In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from time import sleep
import csv
import pandas
import numpy as np
import unidecode

In [2]:
# Función descargaPaginaWeb que permite recuperar la información correspondiente a la respuesta de la petición. 
# url: corresponde con la url del sitio donde se hará el web scraping
def descargaPaginaWeb(url):
    driver = webdriver.Firefox()
    driver.get(url)
    return driver

# Función extraccion_datos que extrae de una página web la información requerida. La función retorna un dataframe con la 
# información de todos los sismos producidos y el nombre de los campos.
# driver: driver utilizado para acceder a los datos de la página web
def extraccion_datos(driver):
    
    # Número de eventos registrados en la página
    filas = len(driver.find_elements_by_xpath("/html/body/div[2]/div[3]/div/div/div/div/div/div/div/div/table/tbody/tr")) + 1
  
    # Número de campos de cada evento
    campos = len(driver.find_elements_by_xpath("/html/body/div[2]/div[3]/div/div/div/div/div/div/div/div/table/tbody/tr[1]/th"))
 
    # Se obtiene el nombre de los campos
    nombre_campos = []
    for c in range(1, campos):
        # obtaining the text from each column of the table
        valor = driver.find_element_by_xpath("/html/body/div[2]/div[3]/div/div/div/div/div/div/div/div/table/tbody/tr[1]/th[" + str(c) + "]").text
        valor = arreglarCampo(valor).strip().replace('\n','')
        nombre_campos.append(valor)
    
    # Se obtienen los valores de cada fila de eventos
    sismos = []
    for f in range(2, filas):
        evento = []
        for c in range(1, campos):
            valor = driver.find_element_by_xpath("/html/body/div[2]/div[3]/div/div/div/div/div/div/div/div/table/tbody/tr[" + str(f) + "]/td[" + str(c) + "]").text
            evento.append(valor)
        sismos.append(evento)
    
    # Se crea un dataframe con la información
    sismos_df = pandas.DataFrame(sismos, columns = nombre_campos)
    return sismos_df

# Función que elimina los acentos y la ñ de los campos de la tabla
def arreglarCampo(campo):
    s = ''.join(c for c in unicodedata.normalize('NFD', campo) if unicodedata.category(c) != 'Mn')
    return s
    
# Función crearArchivoCSVDesdeDF que crea un archivo csv a partir de una lista de diccionarios.
# dataFrame: data frame con la información
# nombreArchivo: nombre del archivo csv
def crearArchivoCSVDesdeDF(dataFrame, nombreArchivo):
    dataFrame.to_csv(nombreArchivo, index = False)


In [3]:
# Función principal
def main():
    driver = descargaPaginaWeb('https://www.ign.es/web/ign/portal/ultimos-terremotos/-/ultimos-terremotos/getAnio')
    sleep(2) # esperar un tiempo como dice la especificación
    sismos_df = extraccion_datos(driver)
    crearArchivoCSVDesdeDF(sismos_df, 'sismos2021.csv')


In [6]:
import unidecode
arreglarCampo('localización y logroño')

'localizacion y logrono'